In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive/training/

Mounted at /content/drive
'new 1.py'		       news-commentary-v9.fr-en.en
 news-commentary-v9.cs-en.cs   news-commentary-v9.fr-en.fr
 news-commentary-v9.cs-en.en   news-commentary-v9.ru-en.en
 news-commentary-v9.de-en.de   news-commentary-v9.ru-en.ru
 news-commentary-v9.de-en.en


In [49]:
with open('/content/drive/MyDrive/training/news-commentary-v9.ru-en.en','rb') as f:
  train_en=f.readlines()
with open('/content/drive/MyDrive/training/news-commentary-v9.ru-en.ru','rb') as f:
  train_fr=f.readlines()
print(len(train_en),len(train_fr))
max=0
for j,i in enumerate(train_en+train_fr):
  if len(i.decode('utf-8').split())>max:
    max=len(i.decode('utf-8').split())
print(max)
MAX_LENGTH=max

165602 165602
171


In [50]:
%matplotlib inline
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [51]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [52]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):    
    s=s.lower().strip()
    s=s.decode('utf-8')
    s = unicodeToAscii(s)
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [53]:
pairs = [[normalizeString(i),normalizeString(j)] for i,j in zip(train_fr,train_en)]
len(pairs)

165602

In [55]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5)
for i,j in kf.split(pairs):
  print(len(i),len(j))
 

132480 33121
132481 33120
132481 33120
132481 33120
132481 33120


In [56]:
#input_lang = Lang('ru')
#output_lang = Lang('eng')
#print("Read %s sentence pairs" % len(pairs))
#pairs = filterPairs(pairs)
#print("Trimmed to %s sentence pairs" % len(pairs))
#num_test = int(len(pairs)/5)
#print(num_test)
#l1=pairs[:num_test]
#l2=pairs[num_test:num_test*2]
#l3=pairs[num_test*2:num_test*3]
#l4=pairs[num_test*3:num_test*4]
#l5=pairs[num_test*4:]
def manage_test_data(test_pairs):
  set_test_eng = set([sent_eng for sent_eng, _ in test_pairs])

  test_pair_dict = {}
  for sent_eng, sent_fre in pairs:
      if sent_eng not in set_test_eng:
          continue 
      elif sent_eng not in test_pair_dict:
          test_pair_dict[sent_eng] = set([sent_fre])
      else:
          test_pair_dict[sent_eng].add(sent_fre)
  return [(sent_eng, list(test_pair_dict[sent_eng])) for sent_eng in test_pair_dict]

#test_pairs = manage_test_data(pairs[132480:])
#test_pairs=test_pairs[:10000]
#train_pairs= pairs[:132480]

#print("Number of test pairs:", len(test_pairs))
#print("Number of train pairs:", len(train_pairs))

In [57]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size,methods, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.methods= methods
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        if methods == "multiplicative":
          print('inside mul')
          self.fc = nn.Linear(hidden_size, hidden_size, bias=False)
      
        elif methods == "additive":
          print('inside add')
          self.fc = nn.Linear(hidden_size, hidden_size, bias=False)
          self.weight = nn.Parameter(torch.FloatTensor(1, hidden_size))

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        #attention in tutorial
        #attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        #attention in lecture slide
        #attn_weights=torch.mm(hidden[0],torch.transpose(encoder_outputs,0,1))
        if self.methods=="multiplicative":
          # For multiplciatve scoring, decoder hidden state is passed through linear layers to introduce a weight matrix
            out = self.fc(hidden[0])
            attn_weights = torch.mm(out,torch.transpose(encoder_outputs,0,1))
          #encoder_outputs.bmm(out.view(1,-1,1)).squeeze(-1)
        elif self.methods=="additive":
          #For additive scoring, decoder hidden state and encoder outputs are concatenated first
            out = torch.tanh(self.fc(hidden[0]+encoder_outputs))
            attn_weights= torch.mm(out,torch.transpose(self.weight,0,1))
          #out.bmm(self.weight.unsqueeze(-1)).squeeze(-1)


        attn_weights=F.softmax(attn_weights.view(1,-1),dim=1)
        
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        #output = F.relu(output)
        output=torch.tanh(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ') if word in lang.word2index]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def trainIters(encoder, decoder, n_iters,train_pairs,input_lang,output_lang, print_every=1000, plot_every=100, learning_rate=0.01):
    #train_pairs=train_pairs,input_lang=input_lang,output_lang=output_lang,
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    print('in train iters',len(train_pairs))
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(train_pairs))
                      for i in range(n_iters)]
    #print(training_pairs[0])
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    #showPlot(plot_losses)

In [58]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [59]:
from nltk.translate.bleu_score import corpus_bleu

def evaluateBleu(encoder, decoder, test_pairs):
    references, candidates = [], []    
    for sents_src, sents_tgt in test_pairs:      
      tgt_words    = [sent_tgt.split(' ') for sent_tgt in sents_tgt]      
      src_words, _ = evaluate(encoder, decoder, sents_src)      
      references.append(tgt_words)
      candidates.append(src_words)      
    score1 = corpus_bleu(references, candidates,weights=(1,0,0,0))
    score2 = corpus_bleu(references, candidates,weights=(0,1,0,0))
    score3 = corpus_bleu(references, candidates,weights=(0,0,1,0))
    return score1,score2,score3

Training and Evaluating

**For BLEU 1,2,3 of cs-en language pair out of 4**

In [23]:
BLEU_1,BLEU_2,BLEU_3=[],[],[]
input_lang = Lang('cs')
output_lang = Lang('eng')
for train_index,test_index in kf.split(pairs):
  train_pairs=[pairs[i] for i in train_index]
  test_pairs=[pairs[i] for i in test_index]
  test_pairs = manage_test_data(test_pairs[:1000]) 
  print("Read %s %s sentence pairs" % (len(train_pairs),len(test_pairs)))
  print("Counting words...")
  for pair in train_pairs:
      input_lang.addSentence(pair[0])
      output_lang.addSentence(pair[1])
  print(input_lang.name, input_lang.n_words)
  print(output_lang.name, output_lang.n_words)
  hidden_size = 256
  encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
  attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words,'multiplicative', dropout_p=0.1).to(device)
  trainIters(encoder1, attn_decoder1, 5000,
             train_pairs=train_pairs,input_lang=input_lang,output_lang=output_lang, 
             print_every=1000)
  b1,b2,b3=evaluateBleu(encoder1, attn_decoder1,test_pairs)
  print('BLEUs :',b1,b2,b3)
  BLEU_1.append(b1)
  BLEU_2.append(b2)
  BLEU_3.append(b3)
print('BLEU1: %s BLEU2: %s BLEU3: %s' % (sum(BLEU_1)/5,sum(BLEU_2)/5,sum(BLEU_3)/5))  

Read 117239 996 sentence pairs
Counting words...
fr 116947
eng 38817
inside mul
in train iters 117239
1m 0s (- 4m 3s) (1000 20%) 6.0456
2m 1s (- 3m 2s) (2000 40%) 5.7166
3m 0s (- 2m 0s) (3000 60%) 5.3675
4m 1s (- 1m 0s) (4000 80%) 5.5010
5m 1s (- 0m 0s) (5000 100%) 5.4592


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEUs : 0.12624313993909492 0.004827104282887336 0.4617006547296093
Read 117239 1000 sentence pairs
Counting words...
fr 129214
eng 41900
inside mul
in train iters 117239
1m 3s (- 4m 13s) (1000 20%) 6.1222
2m 6s (- 3m 9s) (2000 40%) 5.7393
3m 9s (- 2m 6s) (3000 60%) 5.4854
4m 14s (- 1m 3s) (4000 80%) 5.4784
5m 18s (- 0m 0s) (5000 100%) 5.4096
BLEUs : 0.02668740731137595 0.0026607714743029022 0.06387143174964977
Read 117239 999 sentence pairs
Counting words...
fr 129214
eng 41900
inside mul
in train iters 117239
1m 3s (- 4m 12s) (1000 20%) 6.1909
2m 6s (- 3m 9s) (2000 40%) 5.6598
3m 8s (- 2m 5s) (3000 60%) 5.3927
4m 10s (- 1m 2s) (4000 80%) 5.2899
5m 13s (- 0m 0s) (5000 100%) 5.4082
BLEUs : 0.06069724963087063 0.0003190448381690099 0.37023380998190436
Read 117239 999 sentence pairs
Counting words...
fr 129214
eng 41900
inside mul
in train iters 117239
1m 3s (- 4m 12s) (1000 20%) 6.1119
2m 7s (- 3m 10s) (2000 40%) 5.7111
3m 10s (- 2m 6s) (3000 60%) 5.5498
4m 15s (- 1m 3s) (4000 80%) 5.51

**For BLEU 1,2,3 of fr-en langiage pair**

In [36]:
BLEU_1,BLEU_2,BLEU_3=[],[],[]
input_lang = Lang('fr')
output_lang = Lang('eng')
for train_index,test_index in kf.split(pairs):
  train_pairs=[pairs[i] for i in train_index]
  test_pairs=[pairs[i] for i in test_index]
  test_pairs = manage_test_data(test_pairs[:1000]) 
  print("Read %s %s sentence pairs" % (len(train_pairs),len(test_pairs)))
  print("Counting words...")
  for pair in train_pairs:
      input_lang.addSentence(pair[0])
      output_lang.addSentence(pair[1])
  print(input_lang.name, input_lang.n_words)
  print(output_lang.name, output_lang.n_words)
  hidden_size = 256
  encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
  attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words,'multiplicative', dropout_p=0.1).to(device)
  trainIters(encoder1, attn_decoder1, 5000,
             train_pairs=train_pairs,input_lang=input_lang,output_lang=output_lang, 
             print_every=1000)
  b1,b2,b3=evaluateBleu(encoder1, attn_decoder1,test_pairs)
  print('BLEUs :',b1,b2,b3)
  BLEU_1.append(b1)
  BLEU_2.append(b2)
  BLEU_3.append(b3)
print('BLEU1: %s BLEU2: %s BLEU3: %s' % (sum(BLEU_1)/5,sum(BLEU_2)/5,sum(BLEU_3)/5))  

Read 146600 989 sentence pairs
Counting words...
fr 57681
eng 42690
inside mul
in train iters 146600
0m 57s (- 3m 49s) (1000 20%) 6.0369
1m 57s (- 2m 55s) (2000 40%) 5.6381
2m 56s (- 1m 57s) (3000 60%) 5.6125
3m 53s (- 0m 58s) (4000 80%) 5.5039
4m 53s (- 0m 0s) (5000 100%) 5.3780


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEUs : 0.15963483124585068 0.0038679305070789664 4.544202415802726e-05
Read 146600 1000 sentence pairs
Counting words...
fr 62401
eng 46072
inside mul
in train iters 146600
1m 3s (- 4m 12s) (1000 20%) 6.2101
2m 6s (- 3m 9s) (2000 40%) 5.7431
3m 7s (- 2m 5s) (3000 60%) 5.6432
4m 9s (- 1m 2s) (4000 80%) 5.4335
5m 11s (- 0m 0s) (5000 100%) 5.5669
BLEUs : 0.09601507500949734 0.00495149974897943 0.00013829270742550363
Read 146600 999 sentence pairs
Counting words...
fr 62401
eng 46072
inside mul
in train iters 146600
1m 1s (- 4m 7s) (1000 20%) 6.1691
2m 3s (- 3m 5s) (2000 40%) 5.6485
3m 6s (- 2m 4s) (3000 60%) 5.5815
4m 8s (- 1m 2s) (4000 80%) 5.5883
5m 8s (- 0m 0s) (5000 100%) 5.4126
BLEUs : 0.06549602592621502 0.002189941840799438 2.8383205801815882e-05
Read 146600 1000 sentence pairs
Counting words...
fr 62401
eng 46072
inside mul
in train iters 146600
1m 2s (- 4m 10s) (1000 20%) 6.2797
2m 4s (- 3m 6s) (2000 40%) 5.7239
3m 6s (- 2m 4s) (3000 60%) 5.6406
4m 8s (- 1m 2s) (4000 80%) 5.4578

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


**For BLEU 1,2,3 of de-en langiage pair**

In [48]:
BLEU_1,BLEU_2,BLEU_3=[],[],[]
input_lang = Lang('de')
output_lang = Lang('eng')
for train_index,test_index in kf.split(pairs):
  train_pairs=[pairs[i] for i in train_index]
  test_pairs=[pairs[i] for i in test_index]
  test_pairs = manage_test_data(test_pairs[:5000]) 
  print("Read %s %s sentence pairs" % (len(train_pairs),len(test_pairs)))
  print("Counting words...")
  for pair in train_pairs:
      input_lang.addSentence(pair[0])
      output_lang.addSentence(pair[1])
  print(input_lang.name, input_lang.n_words)
  print(output_lang.name, output_lang.n_words)
  hidden_size = 256
  encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
  attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words,'multiplicative', dropout_p=0.1).to(device)
  trainIters(encoder1, attn_decoder1, 5000,
             train_pairs=train_pairs,input_lang=input_lang,output_lang=output_lang, 
             print_every=1000)
  b1,b2,b3=evaluateBleu(encoder1, attn_decoder1,test_pairs)
  print('BLEUs :',b1,b2,b3)
  BLEU_1.append(b1)
  BLEU_2.append(b2)
  BLEU_3.append(b3)
print('BLEU1: %s BLEU2: %s BLEU3: %s' % (sum(BLEU_1)/5,sum(BLEU_2)/5,sum(BLEU_3)/5))  

Read 161029 4997 sentence pairs
Counting words...
de 118742
eng 44415
inside mul
in train iters 161029
1m 7s (- 4m 30s) (1000 20%) 6.2478
2m 14s (- 3m 22s) (2000 40%) 5.8284
3m 23s (- 2m 15s) (3000 60%) 5.6818
4m 31s (- 1m 7s) (4000 80%) 5.5931
5m 40s (- 0m 0s) (5000 100%) 5.6142


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEUs : 0.046856313106810994 0.0016898445512174782 0.40819133937186625
Read 161029 4990 sentence pairs
Counting words...
de 133183
eng 47972
inside mul
in train iters 161029
1m 11s (- 4m 47s) (1000 20%) 6.1686
2m 22s (- 3m 34s) (2000 40%) 5.7832
3m 33s (- 2m 22s) (3000 60%) 5.6125
4m 43s (- 1m 10s) (4000 80%) 5.3924
5m 54s (- 0m 0s) (5000 100%) 5.4717


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEUs : 0.1286137622457288 0.0016582568084202132 1.9031253192130894e-05
Read 161030 4995 sentence pairs
Counting words...
de 133183
eng 47972
inside mul
in train iters 161030
1m 10s (- 4m 40s) (1000 20%) 6.0203
2m 20s (- 3m 30s) (2000 40%) 5.6432
3m 29s (- 2m 19s) (3000 60%) 5.4550
4m 39s (- 1m 9s) (4000 80%) 5.4277
5m 50s (- 0m 0s) (5000 100%) 5.5369
BLEUs : 0.1483992765731058 0.006267119017290033 0.0004879219753179818
Read 161030 4999 sentence pairs
Counting words...
de 133183
eng 47972
inside mul
in train iters 161030
1m 11s (- 4m 46s) (1000 20%) 6.1157
2m 22s (- 3m 34s) (2000 40%) 5.8088
3m 34s (- 2m 23s) (3000 60%) 5.5621
4m 45s (- 1m 11s) (4000 80%) 5.4785
5m 55s (- 0m 0s) (5000 100%) 5.3432
BLEUs : 0.07082247133730094 0.006588834667636186 6.192276903863507e-06
Read 161030 4979 sentence pairs
Counting words...
de 133183
eng 47972
inside mul
in train iters 161030
1m 11s (- 4m 47s) (1000 20%) 6.1592
2m 22s (- 3m 34s) (2000 40%) 5.7135
3m 34s (- 2m 22s) (3000 60%) 5.5641
4m 43s (- 1

**BLEU 1,2,3 for ru-en pairs**

In [60]:
BLEU_1,BLEU_2,BLEU_3=[],[],[]
input_lang = Lang('ru')
output_lang = Lang('eng')
for train_index,test_index in kf.split(pairs):
  train_pairs=[pairs[i] for i in train_index]
  test_pairs=[pairs[i] for i in test_index]
  test_pairs = manage_test_data(test_pairs[:5000]) 
  print("Read %s %s sentence pairs" % (len(train_pairs),len(test_pairs)))
  print("Counting words...")
  for pair in train_pairs:
      input_lang.addSentence(pair[0])
      output_lang.addSentence(pair[1])
  print(input_lang.name, input_lang.n_words)
  print(output_lang.name, output_lang.n_words)
  hidden_size = 256
  encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
  attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words,'multiplicative', dropout_p=0.1).to(device)
  trainIters(encoder1, attn_decoder1, 5000,
             train_pairs=train_pairs,input_lang=input_lang,output_lang=output_lang, 
             print_every=1000)
  b1,b2,b3=evaluateBleu(encoder1, attn_decoder1,test_pairs)
  print('BLEUs :',b1,b2,b3)
  BLEU_1.append(b1)
  BLEU_2.append(b2)
  BLEU_3.append(b3)
print('BLEU1: %s BLEU2: %s BLEU3: %s' % (sum(BLEU_1)/5,sum(BLEU_2)/5,sum(BLEU_3)/5))  

Read 132480 181 sentence pairs
Counting words...
ru 3844
eng 41694
inside mul
in train iters 132480
0m 37s (- 2m 28s) (1000 20%) 6.0691
1m 14s (- 1m 51s) (2000 40%) 5.6909
1m 50s (- 1m 13s) (3000 60%) 5.5573
2m 25s (- 0m 36s) (4000 80%) 5.4642
3m 2s (- 0m 0s) (5000 100%) 5.4603


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEUs : 0.12071424347278437 0.02027952596621585 0.001296648349968136
Read 132481 157 sentence pairs
Counting words...
ru 4762
eng 45120
inside mul
in train iters 132481
0m 37s (- 2m 29s) (1000 20%) 6.0921
1m 16s (- 1m 54s) (2000 40%) 5.6257
1m 54s (- 1m 16s) (3000 60%) 5.5415
2m 31s (- 0m 37s) (4000 80%) 5.3768
3m 8s (- 0m 0s) (5000 100%) 5.4384
BLEUs : 0.08289238862820737 0.008617824936419907 0.000178033480730686
Read 132481 162 sentence pairs
Counting words...
ru 4762
eng 45120
inside mul
in train iters 132481
0m 37s (- 2m 31s) (1000 20%) 6.0266
1m 15s (- 1m 53s) (2000 40%) 5.6261
1m 51s (- 1m 14s) (3000 60%) 5.4517
2m 29s (- 0m 37s) (4000 80%) 5.4973
3m 6s (- 0m 0s) (5000 100%) 5.3838
BLEUs : 0.2390458828018785 0.031594575298347256 0.000557056583389049
Read 132481 176 sentence pairs
Counting words...
ru 4762
eng 45120
inside mul
in train iters 132481
0m 38s (- 2m 33s) (1000 20%) 6.1779
1m 15s (- 1m 53s) (2000 40%) 5.5778
1m 53s (- 1m 15s) (3000 60%) 5.5343
2m 31s (- 0m 37s) (4000 80

In [55]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
trainIters(encoder1, attn_decoder1, 5000,
           train_pairs=train_pairs,input_lang=input_lang,output_lang=output_lang, 
           print_every=1000)
beam1,beam2,beam3 = evaluateBleu_beam_search(encoder1,attn_decoder1,3)
print('beam search :', beam1,beam2,beam3)

in train iters 132480
0m 45s (- 3m 0s) (1000 20%) 6.0962
1m 31s (- 2m 17s) (2000 40%) 5.7634
2m 15s (- 1m 30s) (3000 60%) 5.4124
2m 59s (- 0m 44s) (4000 80%) 5.2970
3m 42s (- 0m 0s) (5000 100%) 5.2483


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


beam search : 0.08307616590917195 0.005428495043861 5.73180779006461e-05
